In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.12.1


In [3]:
import pickle
import pandas as pd

In [25]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [26]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [32]:
input_file = "../data/yellow_tripdata_2023-03.parquet"
output_file = f'output/yellow_tripdata_2023-03.parquet'
df = read_data(input_file)

In [33]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [34]:
y_pred.std()

np.float64(6.247488852238703)

In [35]:
df['ride_id'] = '2023/03_' + df.index.astype('str')

In [37]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)